In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from graphviz import Source
from IPython.display import SVG
from sklearn.tree import export_graphviz
from sklearn.neighbors import NearestNeighbors




In [2]:
# Comma-separated values
phish = pd.read_csv('PhishingData.csv')

# Rename 'num' column to 'disease'
# phish = phish.rename({'num':'disease'}, axis=1)
# display(phish.disease.unique())
# phish['disease'] = phish.disease.apply(lambda x: min(x, 1))

# create a list of feature columns
# feature_cols = ['age','sex','cp','trestbps','chol']
feature_cols = ['SFH','popUpWidnow','SSLfinal_State','Request_URL','URL_of_Anchor','web_traffic','URL_Length','age_of_domain','having_IP_Address']

# define X and y
X = phish[feature_cols]
y = phish.Result

# fit a classification tree with max_depth=3 on all data
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

# # create a Graphviz file
# with open("phish.dot", 'w') as f:
#     f = export_graphviz(treeclf, out_file=f, feature_names=feature_cols, class_names=['0', '1'], filled = True)

graph = Source(tree.export_graphviz(treeclf, out_file=None,
                                    feature_names=feature_cols,
#                                     class_names=['0', '1'], filled = True))
                                    class_names=['0', '1', '2', '3', '4'], filled = True))
svg = SVG(graph.pipe(format='svg'))
display(svg)

# Get a random sample to see how it does
n = phish.shape[0]
# i = random.randint(0, n)
# i = 254 # an interesting case
i = 15 # an interesting case
sample = phish.iloc[i]
print('Sample patient {} to test on. Tree classified as {}'.format(i, treeclf.predict([sample[feature_cols]])))
display(sample)

ExecutableNotFound: failed to execute ['dot', '-Kdot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH